<a href="https://colab.research.google.com/github/amplabs-ai/amplabs/blob/main/python/AmpLabs_Example_Basic_Plotting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AmpLabs Example





In this demo you will learn how to:

1. Upload/Download various types of battery data from **AmpLabs JSON API**
2. Perform simple filtering on battery data

**Pandas** is an easy-to-use data structures and data analysis tools for the Python programming language. In Pandas, we can import data from various file formats like JSON, SQL, Microsoft Excel, etc. When working with tabular data, such as data stored in spreadsheets or databases, pandas will help you to explore, clean, and process your data. In pandas, a data table is called a **Data Frame**.

Note: Pandas is built off of another library called **NumPy**. 


# Key Terms

**API** Application Programming Interface is a set of definitions and protocols for building and integrating application software. AmpLabs provides an API to help you access and control your data.

**JavaScript Object Notation** is a lightweight data-interchange format. It is easy for humans to read and write. It is easy for machines to parse and generate. Win/Win

[JSON](https://www.json.org/json-en.html) is commonly used as an output format from websites. AmpLabs provides a JSON API for your Battery Data. Records for your battery data look like the following: 

``` JSON
{
  "detail": "Records Retrieved", 
  "records": [
    {
      "Charge_Capacity (Ah)": 2.563, 
      "Charge_Energy (Wh)": 10.029, 
      "Cycle_Index": 1, 
      "Discharge_Capacity (Ah)": 2.709, 
      "Discharge_Energy (Wh)": 9.424, 
      "End_Time": null, 
      "Max_Current (A)": 1.496, 
      "Max_Voltage (V)": 4.2, 
      "Min_Current (A)": -1.503, 
      "Min_Voltage (V)": 1.999, 
      "Start_Time": null, 
      "Test_Time (s)": 14644.703
    },
    "status": 200
}
```

**Data Frame**. A DataFrame is a 2-dimensional labeled data structure with columns of potentially different types. You can think of it like a spreadsheet or SQL table, or a dict of Series objects. It is generally the most commonly used pandas object. Like Series, DataFrame accepts many different kinds of input:
[Dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)



``` Python
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
```

In [21]:
import sys
!{sys.executable} -m pip install pandas matplotlib seaborn

You should consider upgrading via the 'c:\Users\Vincent\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Import the libraries we downloaded into the Python environment

In [22]:
# Useful for fetching data from the web 

import json
import urllib.error
import urllib.request
from urllib.parse import urlencode
import requests
import gzip
import json
from fileinput import filename
import shutil
import time
import matplotlib.pyplot as plt
import seaborn as sns

# PyData Libraries

import pandas as pd
import numpy as np

# Define Helper Functions

The following section defines functions that will help us **read** data from and **write** data to the AmpLabs JSON API. 



## Functions to help Upload Data to AmpLabs




In [23]:
# Function used to fetch cycle data from AmpLabs
def get_amplabs_cycledata(user_token, cell_id):
    url = "http://www.amplabs.ai/download/cells/cycle_data_json?cell_id={}".format(cell_id.replace('/','%2F'))
    httprequest = urllib.request.Request( url, method="GET")
    httprequest.add_header("Authorization", "Bearer {}".format(user_token))
    try:
        with urllib.request.urlopen(httprequest) as httpresponse:
            res = json.loads(httpresponse.read())
            with urllib.request.urlopen(urllib.request.Request(res["response_url"],method="GET")) as response:
                response = gzip.decompress(response.read())
                return json.loads(response.decode()), 1
    except urllib.error.HTTPError as e:
        print(e)
    return None, 0

# Function used to fetch timeseries data from AmpLabs
def get_amplabs_timeseriesdata(user_token, cell_id):
    url = "http://www.amplabs.ai/download/cells/cycle_timeseries_json?cell_id={}".format(cell_id.replace('/','%2F'))
    httprequest = urllib.request.Request(
            url, method="GET"
        )
    httprequest.add_header("Authorization", "Bearer {}".format(user_token))

    try:
        with urllib.request.urlopen(httprequest) as httpresponse:
            res = json.loads(httpresponse.read())
            with urllib.request.urlopen(urllib.request.Request(res["response_url"],method="GET")) as response:
                response = gzip.decompress(response.read())
                return json.loads(response.decode()) , 1
    except urllib.error.HTTPError as e:
        print(e)
    return None, 0


# Function used to fetch test meta data from AmpLabs
def get_amplabs_meta(user_token, cell_id):
    url = 'https://www.amplabs.ai/cells/tests/cycle/metaWithId?cell_id={}'.format(cell_id.replace('/','%2F'))
    print(url)
    httprequest = urllib.request.Request( url, method="GET")
    httprequest.add_header("Authorization", "Bearer {}".format(user_token))
    try:
        with urllib.request.urlopen(httprequest) as httpresponse:
          response = json.loads(httpresponse.read())
          return response, 1
    except urllib.error.HTTPError as e:
        print(e)
    return None, 0


**Check your understanding**
1. What is JSON? 

2. Is JSON machine readable?

3. What is a GET Request?

In [24]:
user_token = "<your-token>"
cell_id = 'your-cell-id'

In [25]:
get_amplabs_meta(user_token, cell_id)

https://www.amplabs.ai/cells/tests/cycle/meta?cell_id=NMC-E4-2


({'detail': 'Records Retrieved', 'records': [[]], 'status': 200}, 1)

# Fetch data and store it into a dictionary of data frames

In [35]:
response, status = get_amplabs_timeseriesdata(user_token, cell_id)
df = pd.DataFrame(response['records'][0])

      capacity_throughput cell_temperature  charge_capacity  charge_energy  \
0                    None             None         0.000000       0.000000   
1                    None             None         0.000000       0.000000   
2                    None             None         0.000000       0.000000   
3                    None             None         0.000000       0.000000   
4                    None             None         0.000000       0.000000   
...                   ...              ...              ...            ...   
13578                None             None         0.001317       0.005581   
13579                None             None         0.001317       0.005581   
13580                None             None         0.001317       0.005581   
13581                None             None         0.000000       0.000000   
13582                None             None         0.000000       0.000000   

      cumulative_charge_capacity cumulative_charge_energy  \
0 

In [27]:
df.head(100)

capacity_throughput cell_temperature  charge_capacity  charge_energy  \
0                 None             None     0.000000e+00   0.000000e+00   
1                 None             None     0.000000e+00   0.000000e+00   
2                 None             None     0.000000e+00   0.000000e+00   
3                 None             None     0.000000e+00   0.000000e+00   
4                 None             None     0.000000e+00   0.000000e+00   
..                 ...              ...              ...            ...   
95                None             None     0.000000e+00   0.000000e+00   
96                None             None     1.097185e-09   3.078421e-09   
97                None             None     3.302777e-09   9.277458e-09   
98                None             None     6.609492e-09   1.859500e-08   
99                None             None     8.816440e-09   2.483053e-08   

   cumulative_charge_capacity cumulative_charge_energy  \
0                        None                     None   
1                        None                     None   
2                        None                     None   
3                        None                     None   
4                        None                     None   
..                        ...                      ...   
95                       None                     None   
96                       None                     None   
97                       None                     None   
98                       None                     None   
99                       None                     None   

   cumulative_discharge_capacity cumulative_discharge_energy   current  \
0                           None                        None  0.000000   
1                           None                        None  0.000000   
2                           None                        None  0.000000   
3                           None                        None  0.000000   
4                           None                        None  0.000000   
..                           ...                         ...       ...   
95                          None                        None  0.000000   
96                          None                        None  0.000254   
97                          None                        None  0.000254   
98                          None                        None  0.000254   
99                          None                        None  0.000254   

   cycle_charge_capacity  ... power step_datapoint_ordinal step_index  \
0                   None  ...  None                   None       None   
1                   None  ...  None                   None       None   
2                   None  ...  None                   None       None   
3                   None  ...  None                   None       None   
4                   None  ...  None                   None       None   
..                   ...  ...   ...                    ...        ...   
95                  None  ...  None                   None       None   
96                  None  ...  None                   None       None   
97                  None  ...  None                   None       None   
98                  None  ...  None                   None       None   
99                  None  ...  None                   None       None   

    step_time  step_type test_datapoint_ordinal test_net_capacity  \
0        None       None                   None              None   
1        None       None                   None              None   
2        None       None                   None              None   
3        None       None                   None              None   
4        None       None                   None              None   
..        ...        ...                    ...               ...   
95       None       None                   None              None   
96       None       None                   None              None   
97       None     

In [36]:
# Download python packages to your system using pip install
import sys
!{sys.executable} -m pip install pandas

#suppressing warnings in Jupyter Notebooks
import warnings
warnings.filterwarnings('ignore')

# Useful for fetching data from the web
import json
import urllib.error
import urllib.request

# PyData Libraries
import pandas as pd

def get_amplabs_data(params):
    url = "https://www.amplabs.ai/download/cells/cycle_data_json?{}".format(params)
    httprequest = urllib.request.Request(
            url, method="GET"
        )
    httprequest.add_header("Authorization", f"Bearer {user_token}")
    try:
        with urllib.request.urlopen(httprequest) as httpresponse:
            res = json.loads(httpresponse.read())
            with urllib.request.urlopen(urllib.request.Request(res["response_url"],method="GET")) as response:
                response = gzip.decompress(response.read())
                return json.loads(response.decode())
    except urllib.error.HTTPError as e:
        print(e)
    return None

#Fetch Data from Amplabs API
params = f"cell_id={cell_id}"
response = get_amplabs_data(params)

if response:
    df_cycle = pd.DataFrame(response['records'][0])
    print("Process complete", df_cycle.info())

You should consider upgrading via the 'c:\Users\Vincent\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [33]:
## Voltage-time
def plot_voltage_time(df, xlim=None, ylim=None):
    """
    DESCRIPTION: Takes in a time series dataframe and plots the cell voltage as a function of time
    PARAMETERS:
        df: pandas dataframe
            Time series data of a cell
        x_lim: None, tuple of floats (float, float)
            The x-axis plot range. If set to None, uses matplotlib's default
        y_lim: None, tuple of floats (float, float)
            The y-axis plot range. If set to None, uses matplotlin's default
    RETURNS: None
    """
    # plot parameters
    plt.rcParams.update({'font.size': 25})
    plt.rcParams.update({'font.family':'Arial'})
    # plot size
    plt.figure(figsize=(8,7))
    # font size for x- and y- axis labels
    fsize = 25  
    # ax = plt.gca()
    plt.plot(df['test_time']/3600, df['voltage'], '-', linewidth=1)
    # labelling x and y axes
    plt.ylabel('Voltage [V]', fontsize=fsize)
    plt.xlabel('Time [hours]', fontsize=fsize)
    if xlim:
        plt.xlim(xlim)
    if ylim:
        plt.ylim(ylim)

    # showing the legend, can set legend size
    # plt.legend(prop={'size': 18}).set_draggable(True)
    plt.tight_layout()
    # Save matplotlib figure as png
    print('--------------------------------------------------------------------')
    plt.show()


In [37]:
print(df['test_time'].head(5))
print(df['voltage'].head(5))

0     300.009133
1     600.017945
2     900.026769
3    1200.035577
4    1500.044394
Name: test_time, dtype: float64
0    2.670928
1    2.656848
2    2.645357
3    2.635808
4    2.626906
Name: voltage, dtype: float64


In [38]:
plot_voltage_time(df)

--------------------------------------------------------------------


In [39]:
## Galvanostatic (Voltage-capacity)
def plot_voltage_capacity(df, figure_save_path=None, cycle_range=range(2,3), xlim=None, ylim=None, color_theme='viridis', molar_mass=0, fig_size=(7,7),\
     show=True, ion="", style="standard"):
    """
    DESCRIPTION: Takes in a time series dataframe and plots the cell capacity as a function of voltage
    PARAMETERS:
        df: pandas dataframe
            a pandas dataframe object which contains data for specific capacity and voltage as each cycle index
        figure_save_path: string
            the path to save the plot 
        system: string 
            the name of the system that is being electrochemically cycled. This is usually specified by the raw data file name
        cycle_range: list of integers
            a zero-index list of cycles indices to plot
        x_lim: None, tuple of floats (float, float)
            The x-axis plot range. If set to None, uses matplotlib's default
        y_lim: None, tuple of floats (float, float)
            The y-axis plot range. If set to None, uses matplotlin's default
        show: Boolean
            whether or not to display the matplotlib plot (default=True)
        molar mass: float
            the molar mass of the active material. Used to calculate number of ion ions inserted/ extracted
        ion: string
            the ion that is being transported. Used as part of the axis label name
    RETURNS: None
    """
    # find the number of curves so that colors can be mapped
    num_curves = 0
    first_index = None
    last_index = None
    for index in cycle_range:
        if index < len(df) and index >= 0:
            if num_curves == 0:
                first_index = index
            else:
                last_index = index
            num_curves += 1
        
    # set colors
    if color_theme:
        # print(num_curves)
        sns.set_palette(color_theme, num_curves)
    # plot parameters
    plt.rcParams.update({'font.size': 25})
    plt.rcParams.update({'font.family':'Arial'})
    # plot size
    border_width=2.5
    plt.figure(figsize=fig_size, num=system, dpi=100)
    plt.rcParams["figure.autolayout"] = True
    plt.rcParams["axes.edgecolor"] = "black"
    plt.rcParams["axes.linewidth"] = border_width
    
    # font size for x- and y- axis labels
    fsize = 25  
    ax = plt.gca()
    tick_len = 5
    ax.tick_params(direction="in", width=border_width, length=tick_len)
    # For each cycle, plot both charge and discharge curves with the same color
    for index in cycle_range:
        # print("Original")
        # print()
        # print(df.shape)
        # print(df.head())
        if index < len(df) and index >= 0:
            # the specific capacity data for discharge curveN
            index_filtered = df[df['Cycle_Index'] == index]
            index_filtered = index_filtered.sort_values('Test_Time (s)')
#            print(index_filtered.shape)
#            print(index_filtered.head())
            discharge_cap = np.array(index_filtered[index_filtered['Current (A)'] < 0 ]['Discharge_Capacity (Ah)'])
            # print(index_filtered)
            discharge_voltage = np.array(index_filtered[index_filtered['Current (A)'] < 0]['Voltage (V)'])
            # the specific capacity data for charge curve
            index_filtered_charge = df[(df['Cycle_Index'] == index)]
            charge_cap = np.array(index_filtered_charge[index_filtered_charge['Current (A)'] > 0 ]['Charge_Capacity (Ah)'])
            charge_voltage = np.array(index_filtered_charge[index_filtered_charge['Current (A)'] > 0 ]['Voltage (V)'])

            # if style == "standard":
            #     discharge_cap = np.array(df[index]['discharge']['Discharge_Capacity'])
            # else:
            #     final_charge_cap = charge_cap[-1] 
            #     # the specific capacity data for discharge curve, altered so that the plot will better show hysterisis
            #     discharge_cap = final_charge_cap - np.array(df[index]['discharge']['Discharge_Capacity'])
            #     # the charge and discharge voltages

            # setting the color to plot
            color = next(ax._get_lines.prop_cycler)['color']

            test_time = np.array(df['test_time'])
            charge_capacity = np.array(df['Charge_Capacity (Ah)'])
            print(df['Charge_Capacity (Ah)'].iloc[3000:3010])
            plt.plot(discharge_cap, discharge_voltage)
            plt.plot(charge_cap, charge_voltage)
            # labelling the cycle
            # if index == first_index or index == last_index:
            #     plt.plot(charge_cap, charge_voltage, '-', color=color,  linewidth=4, label='Cycle %s'%(index+1))
            # else:
            #     plt.plot(charge_cap, charge_voltage, '-', color=color,  linewidth=4)
            # # plotting the cycle
            # plt.plot(discharge_cap, discharge_voltage, '-', color=color, linewidth=4)
    # labelling x and y axes
    plt.ylabel('Voltage [V]', fontsize=fsize)
    plt.xlabel('Capacity [mAh/g]', fontsize=fsize)
    if xlim:
        plt.xlim(xlim)
    if ylim:
        plt.ylim(ylim)
    # plt.xlim((-5, 110))
    
    # can manually set the x_ticks (this is where labels will be placed, in terms of specific capacity of active material):
    # ax.set_xticks([-10, 0, 10, 20, 30, 40, 50, 60])

    # In order to have a secondary x axis that shows the amount of ion inserted/ extracted, two functions need to be defined:
    # one to convert specific capacity to ion exchanged, and a reverse function to convert ion exhanged to specific capacity

    # given the molar mass, the specific capacity that corresponds to the insertion of one ion (Li+ or Na+) ion
    # formula: single_ion_capacity [mAh/g] =  = 1/(molar mass)*(faraday's constant)*(coulomb to mAh conversion)
    # dimensional analysis: 1/[mol/g]*[C/mol]*[mAh/C] = [mAh/g]
    # single_ion_capacity = 1/molar_mass*96500*0.2777

    # # function to convert active material specific capacity to alkali ions inserted/ extracted
    # def active_to_ion(capacity):
    #     return capacity/single_ion_capacity

    # # function to convert alkali ions inserted/ extracted to active material specific capacity
    # def ion_to_active(capacity):
    #     return capacity*single_ion_capacity

    # creates a secondary axis showing alkali ions inserted/ extracted using the two functions defined above
    # ax2 = ax.secondary_xaxis('top', functions=(active_to_ion, ion_to_active))
    # ax2.set_xlabel('{}$^+$ ions inserted/ extracted'.format(ion))
    # ax2.tick_params(direction="in", width=border_width, length=tick_len)

    # A way to manually set secondary axis ticks... not recommended
    # xticks = ax.get_xticks()
    # ax2.set_xticks(xticks)
    # ticklabels = [0, 0.5, 1.0, 1.5, 2, 2.5, 3]
    # ax2.set_xticks([x*single_ion_capacity for x in ticklabels])
    # ax2.set_xticklabels(ticklabels)

    # showing the legend, can set legend size
    plt.legend(prop={'size': 18}, frameon=False).set_draggable(True)
    plt.tight_layout()
    plt.show()
    # Save matplotlib figure as png


In [20]:
plotEchem(df)

IndexError: list index out of range